In [1]:
from mmdeploy_runtime import Detector, PoseDetector
import cv2
import numpy as np
import time
from copy import deepcopy 
# from Pose.Hrnet import Hrnet
# from Pose.Yolov7 import Yolov7

In [ ]:
import os.path as osp
import os
def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)

            cv2.imwrite(frame_path, frame)
            cnt += 1
            
    return frame_paths

extract_frame('Video_Fall1/VIDEO1_2.mp4')

In [2]:
# hrnet = Hrnet(engine_path='Pose/Hrnet48_fp32.trt')
# hrnet.get_fps()
# hrnet.destory()
# yolov7 = Yolov7(engine_path='Pose/yolov7_fp16.trt')
# yolov7.get_fps()
# yolov7.destory()

94.00793428013685 FPS
171.42661225308325 FPS


In [3]:
# img = cv2.imread('mmpose/tests/data/coco/000000196141.jpg')
# det_result = yolov7.inference(img)
# pose_result = hrnet.inference_from_bbox(img,det_result)

In [3]:
def visualize(frame, anno:dict, thr=0.5, resize=None):
    keypoints = anno['keypoints']
    bboxes = anno['bbox']
    skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
                (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
                (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
    palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
               (255, 153, 255), (153, 204, 255), (255, 102, 255),
               (255, 51, 255), (102, 178, 255),
               (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
               (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
               (0, 0, 255), (255, 0, 0), (255, 255, 255)]
    link_color = [
        0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
    ]
    point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]
    img = deepcopy(frame)
    scores = keypoints[..., 2]
    keypoints = (keypoints[..., :2]).astype(int)
    if resize:
        scale = resize / max(frame.shape[0], frame.shape[1])

       
        keypoints = (keypoints[..., :2] * scale).astype(int)

        img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
    for kpts, score in zip(keypoints, scores):
        shows = [0] * len(kpts)
        for (u, v), color in zip(skeleton, link_color):
            if score[u] > thr and score[v] > thr:
                cv2.line(img, kpts[u], tuple(kpts[v]), palette[color], 1,
                         cv2.LINE_AA)
                shows[u] = shows[v] = 1
        for kpt, show, color, bbox in zip(kpts, shows, point_color, bboxes):
            if show:
                cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])) , (0,255,0), 1)
    return img

# def visualize(frame, anno, thr=0.5, resize=1280):
#     keypoints = anno['keypoints']
# #     bboxes = anno['bbox']
#     skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
#                 (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
#                 (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
#     palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
#                (255, 153, 255), (153, 204, 255), (255, 102, 255),
#                (255, 51, 255), (102, 178, 255),
#                (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
#                (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
#                (0, 0, 255), (255, 0, 0), (255, 255, 255)]
#     link_color = [
#         0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
#     ]
#     point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]

#     scale = resize / max(frame.shape[0], frame.shape[1])

#     scores = keypoints[..., 2]
#     keypoints = (keypoints[..., :2] * scale).astype(int)

#     img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
#     for kpts, score in zip(keypoints, scores):
#         show = [0] * len(kpts)
#         for (u, v), color in zip(skeleton, link_color):
#             if score[u] > thr and score[v] > thr:
#                 cv2.line(img, kpts[u], tuple(kpts[v]), palette[color], 1,
#                          cv2.LINE_AA)
#                 show[u] = show[v] = 1
#         for kpt, show, color in zip(kpts, show, point_color):
#             if show:
#                 cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
#     return img

In [2]:
det_model = Detector('rtmpose-trt/rtmdet-nano/','cuda')
pose_model = PoseDetector('rtmpose-trt/rtmpose-m/','cuda')

In [28]:
img = cv2.imread('temp/img_00001.jpg')
bboxes, labels, _ = det_model(img)
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.7)
bboxes1 = bboxes[keep]

In [31]:
len(bboxes1)

0

In [15]:
def visualize(frame, anno:dict, thr=0.5, resize=None):
    if anno is None:
        if resize:
            scale = resize / max(frame.shape[0], frame.shape[1])
            frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
        return frame
    keypoints = anno['keypoints']
    bboxes = anno['bbox']
    skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
                (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
                (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
    palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
               (255, 153, 255), (153, 204, 255), (255, 102, 255),
               (255, 51, 255), (102, 178, 255),
               (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
               (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
               (0, 0, 255), (255, 0, 0), (255, 255, 255)]
    link_color = [
        0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
    ]
    point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]
    img = deepcopy(frame)
    scores = keypoints[..., 2]
    keypoints = (keypoints[..., :2]).astype(int)
    if resize:
        scale = resize / max(frame.shape[0], frame.shape[1])

       
        keypoints = (keypoints[..., :2] * scale).astype(int)
        for i,b in enumerate(bboxes):
            bboxes[i] = b*scale
        img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
    for kpts, score,bbox in zip(keypoints, scores, bboxes):
        shows = [0] * len(kpts)
        for (u, v), color in zip(skeleton, link_color):
            if score[u] > thr and score[v] > thr:
                cv2.line(img, tuple(kpts[u]), tuple(kpts[v]), palette[color], 1,
                         cv2.LINE_AA)
                shows[u] = shows[v] = 1
        for kpt, show, color in zip(kpts, shows, point_color):
            if show:
               cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])) , (0,255,0), 1)
    return img

In [16]:
# def visualize(frame, anno:dict, thr=0.5, resize=None):
#     if anno is not None:
#         keypoints = anno['keypoints']
#         bboxes = anno['bbox']
#     if resize:
#         scale = resize / max(frame.shape[0], frame.shape[1])
#         img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
#         if anno is None:
#             return img
#         keypoints = (keypoints[..., :2] * scale).astype(int)
#         for i,b in enumerate(bboxes):
#             bboxes[i] = b*scale
#     if anno is None:
#         return frame
#     skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
#                 (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
#                 (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
#     palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
#                (255, 153, 255), (153, 204, 255), (255, 102, 255),
#                (255, 51, 255), (102, 178, 255),
#                (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
#                (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
#                (0, 0, 255), (255, 0, 0), (255, 255, 255)]
#     link_color = [
#         0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
#     ]
#     point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]
#     # img = deepcopy(frame)
#     scores = keypoints[..., 2]
#     keypoints = (keypoints[..., :2]).astype(int)
#     for kpts, score,bbox in zip(keypoints, scores, bboxes):
#         shows = [0] * len(kpts)
#         for (u, v), color in zip(skeleton, link_color):
#             if score[u] > thr and score[v] > thr:
#                 cv2.line(img, tuple(kpts[u]), tuple(kpts[v]), palette[color], 1,
#                          cv2.LINE_AA)
#                 shows[u] = shows[v] = 1
#         for kpt, show, color in zip(kpts, shows, point_color):
#             if show:
#                cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
#         cv2.rectangle(img, (int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])) , (0,255,0), 1)
#     return img

img = cv2.imread('Image/img_00001.jpg')
bboxes, labels, _ = det_model(img)
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.7)
bboxes = bboxes[keep]
if len(bboxes) == 0:
    anno = None
else:
    bbox = []
    score_det = []
    for item in bboxes:
        bbox.append(item[...,:4])
        score_det.append(item[4])
    keypoints = pose_model(img,bbox)
    anno = dict()
    anno['bbox'] = bbox
    anno['keypoints'] = keypoints
img_o = visualize(img,anno,resize=640)
cv2.imshow('',img_o)
cv2.waitKey(-1)

-1

In [14]:
len(bbox), len(keypoints)

(3, 3)

In [50]:
a = [i for i in range(4)]
b = [i+1 for i in range(4)]
c = [i+2 for i in range(4)]
d = [i+3 for i in range(4)]

In [51]:
for aa,bb,cc,dd in zip(a,b,c,d):
    print(aa,bb,cc,dd)

0 1 2 3
1 2 3 4
2 3 4 5
3 4 5 6


In [48]:
result = det_model(img)

In [81]:
bboxes, labels, _ = result

In [51]:
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.6)
bboxes1 = bboxes[keep,:4]

In [42]:
bboxes2 = bboxes[keep]

In [46]:
# img = cv2.imread('Image/img_00001.jpg')
bboxes, labels, _ = det_model(img)
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.7)
bboxes = bboxes[keep]
bbox = []
score_det = []
for item in bboxes:
    bbox.append(item[...,:4])
    score_det.append(item[4])
keypoints = pose_model(img,bbox)
anno = dict()
anno['bbox'] = bbox
anno['keypoints'] = keypoints
img_o = visualize(img,anno,thr=0.5)
cv2.imshow('',img_o)
cv2.waitKey(-1)

-1

In [40]:
img_o.shape

(858, 1280, 3)

In [24]:
anno

{'bbox': [array([ 36.980553 ,   1.3494898, 199.61458  , 354.34442  ], dtype=float32)],
 'keypoints': array([[[123.56154   ,  57.40497   ,   0.8976282 ],
         [130.45387   ,  48.76245   ,   0.967244  ],
         [117.53078   ,  49.62668   ,   0.9553641 ],
         [141.65382   ,  40.98416   ,   0.95445657],
         [108.05385   ,  41.84842   ,   0.9428457 ],
         [161.46925   ,  66.047516  ,   0.8510432 ],
         [ 91.68465   ,  77.28281   ,   0.8237685 ],
         [151.99232   , 129.138     ,   0.89654493],
         [111.500015  , 137.78055   ,   0.85989505],
         [144.23848   , 177.53622   ,   0.7529628 ],
         [132.1769    , 187.90726   ,   0.83874285],
         [151.13075   , 165.43665   ,   0.7373272 ],
         [103.74617   , 165.43665   ,   0.706004  ],
         [170.94618   , 238.03394   ,   0.84957266],
         [ 80.484604  , 235.44116   ,   0.85689986],
         [182.14613   , 322.73074   ,   0.9464668 ],
         [ 55.500015  , 318.40945   ,   0.9652223 ]]

In [20]:
bboxes, labels, _ = det_model(img)
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.7)
bbox = []
score_det = []
bboxes = bboxes[keep]
# bboxes = bboxes[0]
bbox.append(bboxes[0,:4])
score_det = bboxes[0,4]
keypoints = pose_model(img,bbox)
anno = dict()
anno['bbox'] = bbox
anno['keypoints'] = keypoints


In [18]:
bbox

In [22]:
# print(f'Hrnet: \n{pose_result}')
print(f'RTM-Pose: \n{anno}')

RTM-Pose: 
{'bbox': [array([245.50363 ,  74.826294, 417.49356 , 367.39346 ], dtype=float32)], 'keypoints': array([[[300.92188   , 112.76367   ,   0.9510821 ],
        [303.78647   , 107.74609   ,   0.9043638 ],
        [300.20575   , 107.0293    ,   0.8778724 ],
        [315.96094   , 111.33008   ,   0.7930349 ],
        [328.85156   , 101.29492   ,   0.6546844 ],
        [325.987     , 143.58594   ,   0.73849547],
        [353.9167    , 114.91406   ,   0.83227235],
        [295.90887   , 167.95703   ,   0.8247163 ],
        [316.6771    , 131.40039   ,   0.71336913],
        [267.9792    , 155.77148   ,   0.8414177 ],
        [278.72137   , 142.15234   ,   0.7866514 ],
        [328.13544   , 218.13281   ,   0.68726873],
        [346.75522   , 213.11523   ,   0.52360314],
        [324.5547    , 296.26367   ,   0.7868002 ],
        [281.58594   , 265.4414    ,   0.8362156 ],
        [391.87244   , 329.23633   ,   0.67278886],
        [287.31512   , 343.57227   ,   0.81489193]]], dtype=f